In [ ]:
# ✔ Dataset load kiya
# ✔ Cleaning checks kiye
# ✔ Data quality score nikala
# ✔ Combined quality score nikala

# Ab next step = cleaned dataset ko RAG pipeline me bhejna.

In [2]:
111+1


112

In [7]:
%pwd

'e:\\MentalHealth__\\resource'

In [8]:
import os
print(os.getcwd())

e:\MentalHealth__\resource


In [9]:
import os
# to get back one folder back
os.chdir("../")

In [10]:
%pwd

'e:\\MentalHealth__'

In [17]:
# Basics Check

In [16]:
import pandas as pd

# Load your CSV
df1 = pd.read_csv("datasets/depression_dataset_reddit_cleaned.csv")
df2 = pd.read_csv("datasets/survey.csv")

# 1. Check shape
print(f"Rows in dataset 1 : {len(df1)}")
print(f"Columns in dataset 1 : {df1.columns.tolist()}")

print(f"Rows in dataset 2 : {len(df2)}")
print(f"Columns in dataset 2 : {df2.columns.tolist()}")

print("--------------------------------------------------------------------------")
# 2. Check for nulls
print(df1.isnull().sum())
print(df2.isnull().sum())

print("--------------------------------------------------------------------------")
# 3. Check duplicates
print(f"Duplicates in datasets 1: {df1.duplicated().sum()}")
print(f"Duplicates in datasets 2: {df2.duplicated().sum()}")

print("---------------------------------------------------------------------------")
# 4.1 Sample data in dataset 1
print(df1.head(2))
# print(df1.sample(10))  # Random samples

print("-------------------------------------------------------------------------")
# 4.2 Sample data in dataset 2
print(df2.head(2))
# print(df2.sample(10))  # Random samples

print("--------------------------------------------------------------------------")
# 5. Text length distribution
df1['text_length'] = df1['clean_text'].str.len()
print(df1['text_length'].describe())

# 5. Text length distribution
df2['text_length'] = df2['comments'].str.len()
print(df2['text_length'].describe())


Rows in dataset 1 : 7731
Columns in dataset 1 : ['clean_text', 'is_depression']
Rows in dataset 2 : 1259
Columns in dataset 2 : ['Timestamp', 'Age', 'Gender', 'Country', 'state', 'self_employed', 'family_history', 'treatment', 'work_interfere', 'no_employees', 'remote_work', 'tech_company', 'benefits', 'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave', 'mental_health_consequence', 'phys_health_consequence', 'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview', 'mental_vs_physical', 'obs_consequence', 'comments']
--------------------------------------------------------------------------
clean_text       0
is_depression    0
dtype: int64
Timestamp                       0
Age                             0
Gender                          0
Country                         0
state                         515
self_employed                  18
family_history                  0
treatment                       0
work_interfere                264
no_em

In [19]:
# Rename text columns
df1 = df1.rename(columns={'clean_text': 'text'})
df2 = df2.rename(columns={'comments': 'text'})


In [ ]:
# 6. Check for incomplete sentences
incomplete = df1[df1['text'].str.endswith('.') == False]
print(f"Incomplete sentences: {len(incomplete)}")

# 7. Check for very short text (likely bad quality)
short_text = df1[df1['text'].str.len() < 50]
print(f"Very short entries: {len(short_text)}")

# 8. Check for special characters/encoding issues
import re
weird_chars = df1[df1['text'].str.contains('[^\x00-\x7F]+', regex=True)]
print(f"Non-ASCII text: {len(weird_chars)}")

# 9. Check for repetitive text
# (Sometimes datasets have duplicate paragraphs)
df1['text_unique'] = df1['text'].drop_duplicates()
print(f"Unique text entries: {len(df1['text_unique'])}")

Incomplete sentences: 7731
Very short entries: 1536
Non-ASCII text: 5
Unique text entries: 7731


In [26]:
# 6. Check for incomplete sentences
incomplete = df2[df2['text'].str.endswith('.') == False]
print(f"Incomplete sentences: {len(incomplete)}")

# 7. Check for very short text (likely bad quality)
short_text = df2[df2['text'].str.len() < 50]
print(f"Very short entries: {len(short_text)}")

# 8. Check for special characters/encoding issues
import re
weird_chars = df2[df2['text'].str.contains('[^\x00-\x7F]+', regex=True, na=False)]
print(f"Non-ASCII text: {len(weird_chars)}")

# 9. Check for repetitive text
# (Sometimes datasets have duplicate paragraphs)
df2['text_unique'] = df2['text'].drop_duplicates()
print(f"Unique text entries: {len(df2['text_unique'])}")

Incomplete sentences: 46
Very short entries: 26
Non-ASCII text: 0
Unique text entries: 1259


In [29]:
def data_quality_score(df, text_column):
    """
    Returns quality score 0-100
    """
    score = 100
    
    # Penalize nulls
    null_pct = df[text_column].isnull().sum() / len(df)
    score -= null_pct * 30
    
    # Penalize duplicates
    dup_pct = df.duplicated().sum() / len(df)
    score -= dup_pct * 20
    
    # Penalize very short text
    short_pct = (df[text_column].str.len() < 100).sum() / len(df)
    score -= short_pct * 20
    
    # Penalize incomplete sentences
    incomplete_pct = (df[text_column].str.endswith('.') == False).sum() / len(df)
    score -= incomplete_pct * 10
    
    return max(0, score)

def quality_rating(score):
    """Return star rating label."""
    if score >= 90:
        return "⭐⭐⭐⭐⭐ Excellent"
    elif score >= 80:
        return "⭐⭐⭐⭐ Very Good"
    elif score >= 70:
        return "⭐⭐⭐ Good"
    elif score >= 60:
        return "⭐⭐ Fair"
    else:
        return "⭐ Poor"


In [30]:
quality1 = data_quality_score(df1, 'text')
print(f"Data Quality Score: {quality1}/100")
print("----------------------------------------------------------------")
quality2 = data_quality_score(df2, 'text')
print(f"Data Quality Score: {quality2}/100")

Data Quality Score: 80.5833656706765/100
----------------------------------------------------------------
Data Quality Score: 72.76409849086576/100


In [32]:
# -----------------------------
# COMBINED SCORE + RATING
# -----------------------------

# Combine both datasets for overall score
combined_df = pd.concat([df1, df2], ignore_index=True)

combined_quality = data_quality_score(combined_df, "text")
combined_rating = quality_rating(combined_quality)

print(f"\nCombined Dataset Quality Score: {combined_quality}/100")
print(f"Combined Rating: {combined_rating}")


Combined Dataset Quality Score: 79.48832035595105/100
Combined Rating: ⭐⭐⭐ Good
